# A free form canvas in Jupyter Lab with Literally Canvas

In [1]:
from dataclasses import dataclass, field
import json, pandas

In [2]:
@dataclass(order=True)
class LiterallyCanvas:
    file: str = None
    data: dict = field(default_factory=dict)
    def __post_init__(self):
        self.file and self.load()

    def load(self): 
        if self.file:
            with open(self.file) as file: self.data.update(json.load(file))
                
    def to_series(self): return pandas.read_json(self.file, typ='Series')
    
    
    def __add__(self, object):
        if isinstance(object, type(self)):
            self.data['shapes'] += object.data['shapes']
        return self
    
    def __sub__(self, object):
        if isinstance(object, type(self)):
            self.data['shapes'] += [object for object in self.data['shapes'] if object not in object.data['shapes']]
        return self

    def _repr_mimebundle_(self, **_):
        return {
            "application/literallycanvas+json": json.dumps(self.data)
        }, {}

# Combining and subtracting canvases should be easy. 

# Shifting should be easy too

In [45]:
    from importlib_resources import path
    from pytest import fixture

In [46]:
    from IPython.display import display

In [47]:
    @fixture
    def reincarnation_canvas():
        with path('deathbeds', 'reincarnation.literallycanvas') as file: return LiterallyCanvas(file)

In [48]:
if __name__ == '__main__': 
    with path('deathbeds', 'reincarnation.literallycanvas') as file:
            display(reincarnation_canvas())

In [49]:
    def _canvas_data_in_the_notebook(reincarnation_canvas):
        from deathbeds import __Literally_canvas_data
        df = pandas.DataFrame(pandas.read_json(__Literally_canvas_data.__file__, typ='Series').loc['cells'])
        canvas = json.loads(df.outputs.apply(pandas.Series).dropna()[0].apply(lambda x: x.get('data', {}).get('application/literallycanvas+json', None)).dropna().iloc[0])
        assert canvas == reincarnation_canvas.data